# Part I. ETL Pipeline for Pre-Processing the Files

## CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f)) 

6821


# Part II. Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor':1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Below are the queries on which we need to build our model

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1 - Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

- First we need to create a table by identifying the primary keys from the given dataset.  

- We want to query the data on session_id and item_in_session so these columns must be included in primary key.

- From the primary keys data is partioned on session_id consider as Partition key and item_in session as a clustering column.



In [8]:
create_table1= "CREATE TABLE IF NOT EXISTS songs_history"
create_table1= create_table1 + ("""(session_id int,item_in_session int,artist text,song_title text,song_length FLOAT,
                PRIMARY KEY(session_id, item_in_session))""")

try:
    session.execute(create_table1)
    print("Table songs_history created")
except Exception as e:
    print(e)
                    

Table songs_history created


## Insert the data into Song_histroy table

- Below, function f is reading the data from CSV file line by line and inserting the data into table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO songs_history(session_id,item_in_session,artist,song_title,song_length)"
        query1 = query1 + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query1, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Result of the Query 

- Below, we have written the select query for which parameters are passed while in session.
- Also used the PrettyTable to display the output of query

In [19]:

select_query1 = "SELECT session_id,item_in_session,artist,song_title,song_length from songs_history WHERE session_id= %s AND item_in_session= %s"

try:
    rows = session.execute(select_query1,(338,4))
except Exception as e:
    print(e)
    
t = PrettyTable (['# Session','# Session Item','Artist','Song Title','Song Length'])
for row in rows:
    t.add_row([row.session_id, row.item_in_session, row.artist, row.song_title, row.song_length])
print(t)

+-----------+----------------+-----------+---------------------------------+--------------------+
| # Session | # Session Item |   Artist  |            Song Title           |    Song Length     |
+-----------+----------------+-----------+---------------------------------+--------------------+
|    338    |       4        | Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+----------------+-----------+---------------------------------+--------------------+


## Query 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- First we need to create a table by identifying the primary keys from the given dataset.  

- We want to query the data on user_id and session_id so these columns must be included in primary key. Also, we want to sort the data by item_in_session so this column will act as a clustering column.

- So, our primary keys will be ((user_id,session_id),item_in_session), where (user_id,session_id) is the partition key and item_in_session is clustering column.


In [20]:
create_table2 = "CREATE TABLE IF NOT EXISTS artists"
create_table2 = create_table2 + """(user_id int, session_id int,item_in_session int, artist_name text, 
                   song_title text, first_name text, last_name text,PRIMARY KEY(user_id,session_id, item_in_session))"""

try:
    session.execute(create_table2)
    print("Table artists created")
except Exception as e:
    print(e)

Table artists created


## Insert the data into Artists table

- Below, function f is reading the data from CSV file line by line and inserting the data into table

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO artists(user_id, session_id,item_in_session,artist_name,song_title , first_name , last_name )"
        query2 = query2 + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query2, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

#### Result of the Query 

- Below, we have written the select query with passing the parameters in session. 
- Also used the PrettyTable to display the output of query

In [22]:
select_query2 = "SELECT artist_name, song_title, first_name, last_name from artists WHERE user_id= %s AND session_id= %s"

try:
    rows = session.execute(select_query2,(10,182))
except Exception as e:
    print(e)

t = PrettyTable(['Artist Name','Song Title','First Name','Last Name'])

for row in rows:
    t.add_row([row.artist_name,row.song_title,row.first_name,row.last_name])
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|    Artist Name    |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


## Query 3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- First we need to create a table by identifying the primary keys from the given dataset.  

- We want to query the data on song title so it should be included in primary key, also we need to use the another column so that to remove the duplicate users for the song. 

- Based on the data we can take user_id as primary key. so we have primary key ((song_title),user_id), where song_title is partition key and user_id as clustering column.


In [23]:
create_table3 = "CREATE TABLE IF NOT EXISTS app_history"
create_table3 = create_table3 + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY ((song_title),user_id))"

try:
    session.execute(create_table3)
    print("Table app_history created")
except Exception as e:
    print(e)       

Table app_history created


## Insert the data into Artists table

- Below, function f is reading the data from CSV file line by line and inserting the data into table

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query3 = "INSERT INTO app_history (song_title, user_id, first_name, last_name)"
        query3 = query3 + "VALUES (%s,%s,%s,%s)"
        session.execute (query3, (line[9],int(line[10]),line[1],line[4]))

#### Result of the Query 

- Below, we have written the select query with passing the parameters in session. 
- Also used the PrettyTable to display the output of query

In [25]:
select_query3 = "SELECT first_name, last_name from app_history WHERE song_title=%s"

try:
    rows = session.execute((select_query3), ('All Hands Against His Own', ))
except Exception as e:
    print(e)

t = PrettyTable(['First name','Last Name'])
for row in rows:
    t.add_row ([row.first_name,row.last_name])
print(t)

+------------+-----------+
| First name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [26]:
session.execute ("DROP TABLE IF EXISTS songs_history")
session.execute ("DROP TABLE IF EXISTS artists")
session.execute ("DROP TABLE IF EXISTS app_history")

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()